# ECE 143 Individual Project
## Analysis of Ad-Hoc Communications Network
**Name**: Shijia Shu

**Student ID**: A99040010


## Problem Statement

You have been asked to help with planning an ad-hoc communications network over a large rectangular region. Each individual tower can monitor a rectangular subsection of a specific
width and height. The main problem is that none of the individual towers can provide
coverage for the entire region of interest. Communications towers are unreliable and are put
up independently and at random. You have no control over where or how big a tower’s
footprint is placed. Importantly, due to technical issues such as cross-talk, no individual
rectangular subsection can have multiple towers providing coverage for it. That is, there can
be no overlap between any pair of rectangular subsections provided by the two respective
towers. In any case, the desire is to maximize the coverage area of any available
communications tower.

The order of when the towers come online is important. Once a tower has acquired its
rectangular section, no subsequent tower can overlap that section. You may assume the
following for this problem:
* All rectangular sections have integer-based corners.
* All rectangular sections must be contained in the overall rectangular footprint.
* The height and width of each rectangular section is sampled from a uniform
distribution.
* Positions of the windows are also determined by uniform random distribution.
* All footprints must be rectangles (not general polygons).
* When a new tower comes online, if its coverage rectangle intersects the pre-existing
composite footprint, then that new tower’s coverage is trimmed such that its
maximum remaining coverage area is retained (see sequential diagram below).

Write a detailed Jupyter notebook that implements a solution to this problem such that the
user can supply the following overall size of desired coverage footprint and then determine
the following:
* Given an overall desired coverage footprint and a sequence of n communications
towers, what is the resulting resolved coverage?
* What is the total area of coverage relative to the desired total coverage area of the
original footprint? That is, are there any gaps in coverage?
* On average, how many communications towers are required before full coverage is
obtained?

## Analysis on the Problem

Approach
* Functions
    * coverage_generation_random() # Generate a list of towers by inputting the desired coverage. 
    * coverage_generation() # Generate the coverage map by inputing the desired coverage and a sequence of n  communication towers. 
        * generate_canvas( ) # To create the canvas based on user input area
        * generate_rand_num( ) # numpy.random.choice()
        * place_tower() # Update the canvas by placing the tower
        * find_max_rect( ) # Find the maximum non-overlappying rectangle on the canva.
    * avg_tower_need( ) # Calculate the average number of towers needed for given desired coverage (question 3)
    

## Solution to the Problem

In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"packages"))
import adhoc
import numpy as np
reload(adhoc)

<module 'adhoc' from '/Users/shijia/Documents/@UCSD/Courses/Spring2018/ECE143/Individual-Project/ECE143-Individual-Project/packages/adhoc.pyc'>

In [2]:
region = adhoc.generate_region(10,8)
region

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [3]:
tower = adhoc.generate_tower(region.shape)
print tower
region_update = adhoc.place_tower(region, tower)
region_update

{'loc': (3, 2), 'length': 6, 'width': 5}


array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [4]:
tower = adhoc.generate_tower(region.shape)
print tower
region_update = adhoc.place_tower(region_update, tower)
region_update

{'loc': (1, 0), 'length': 4, 'width': 5}


array([[0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 2, 2, 2, 1, 1, 0],
       [1, 1, 2, 2, 2, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
test = region_update.copy()

print test
#adhoc.trim_tower(test, tower)

In [ ]:
reload(adhoc)
#print test
trimmed_tower = adhoc.trim_tower(test, tower)

In [ ]:
reload(adhoc)
A = adhoc.update_tower_coverage(region, trimmed_tower)
print A


In [ ]:
lala = [1,2,3]
baba = lala
baba.append(4)
print id(lala) == id(baba)
print "lala = ", lala
print "baba = ", baba

## Testing Cases

####  Random Generation

In [13]:
reload(adhoc)
region = adhoc.generate_region(10,8)
region

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

#### Place the new tower and trim the coverage to provide the maximum coverage

In [17]:
# Repeat till the whole region has been covered with communication tower
tower = adhoc.generate_tower(region.shape)
print "Tower Generation: \n", tower

region_update = adhoc.place_tower(region, tower)
print "The region after adding tower: \n", region_update

tower_area, tower_loc_new = adhoc.trim_tower(region_update, tower, show_coverage_update=True)


region = adhoc.place_tower(region, tower_loc_new)
print "The updated region coverage: \n", region

Tower Generation: 
{'loc': (3, 7), 'length': 1, 'width': 1}
The region after adding tower: 
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 2]
 [0 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1]
 [0 1 1 1 1 0 0 0]
 [0 1 1 1 1 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0]]
Trimed Tower Area:  0
The updated region coverage: 
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1]
 [0 1 1 1 1 0 0 0]
 [0 1 1 1 1 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0]]


In [ ]:
region = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                   [0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                   [0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                   [0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                   [0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0,0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
region

## Conclusion